In [1]:
# !pip install langchain==0.0.346
# !pip install newspaper3k==0.2.8
# !pip install pandas==2.1.3
# !pip install deltalake==0.14.0
# !pip install tabulate==0.9.0
# !pip show 

In [2]:
import sys
sys.path.append('..')

from langchain.document_loaders import NewsURLLoader
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
pd.set_option('display.expand_frame_repr', False)
from tabulate import tabulate

from deltalake import DeltaTable

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
def load_news(urls):
    newsloader = NewsURLLoader(urls=urls, nlp=True)
    data = newsloader.load()
    return data

def print_df(df):
    return tabulate(df, headers='keys', tablefmt='html')

In [4]:
urls = [
    "https://www.nbcnews.com/health/health-news/makers-ozempic-mounjaro-sued-stomach-paralysis-claims-rcna97819",
    "https://www.nbcnews.com/news/us-news/harambe-gorilla-killed-cincinnati-zoo-had-pay-price-experts-n583146",
]

In [5]:
data = load_news(urls)

In [6]:

def convert_document_to_dict(doc):
    result = doc.metadata # json object
    result['page_content'] = doc.page_content # string
    return result

df = pd.DataFrame(data=[convert_document_to_dict(x) for x in data])
print_df(df)

'<table>\n<thead>\n<tr><th style="text-align: right;">  </th><th>title                                                                     </th><th>link                                                                                                      </th><th>authors                                                                                                                                                                                                                                                                                                                                                              </th><th>language  </th><th>description                                                                                                                                                                                           </th><th>publish_date  </th><th>keywords                                                                                                                                                                                  </th><th>summary  </th><th>page_content  </th></tr>\n</thead>\n<tbody>\n<tr><td style="text-align: right;"> 0</td><td>Makers of Ozempic and Mounjaro sued over &#x27;stomach paralysis&#x27; claims       </td><td>https://www.nbcnews.com/health/health-news/makers-ozempic-mounjaro-sued-stomach-paralysis-claims-rcna97819</td><td>[&#x27;Https&#x27;, &#x27;Media-Cldnry.S-Nbcnews.Com Image Upload Newscms&#x27;, &#x27;Berkeley Lovelace Jr.&#x27;, &#x27;Berkeley Lovelace Jr. Is A Health&#x27;, &#x27;Medical Reporter For Nbc News. He Covers The Food&#x27;, &#x27;Drug Administration&#x27;, &#x27;With A Special Focus On Covid Vaccines&#x27;, &#x27;Prescription Drug Pricing&#x27;, &#x27;Health Care. He Previously Covered The Biotech&#x27;, &#x27;Pharmaceutical Industry With Cnbc.&#x27;]</td><td>en        </td><td>Ozempic and Mounjaro are the subject of a lawsuit alleging that the drugs can cause severe stomach paralysis and that drugmakers Novo Nordisk and Eli Lilly failed to adequately warn consumers about.</td><td>              </td><td>[&#x27;drugs&#x27;, &#x27;ozempic&#x27;, &#x27;according&#x27;, &#x27;caused&#x27;, &#x27;mounjaro&#x27;, &#x27;makers&#x27;, &#x27;diabetes&#x27;, &#x27;sued&#x27;, &#x27;claims&#x27;, &#x27;gastroparesis&#x27;, &#x27;medications&#x27;, &#x27;stomach&#x27;, &#x27;pennock&#x27;, &#x27;paralysis&#x27;, &#x27;condition&#x27;, &#x27;lawsuit&#x27;]</td><td>The drugmakers Novo Nordisk and Eli Lilly failed to adequately warn patients about the possible risk of severe stomach problems associated with their blockbuster drugs Ozempic and Mounjaro, according to a lawsuit filed Wednesday.\nOzempic and Mounjaro — both part of a new class of medicines known as GLP-1 agonists — help people with Type 2 diabetes manage their blood sugar levels.\nGLP-1 medications work, in part, by slowing how quickly food moves through the stomach, which can lead to a person feeling fuller longer.\nGastroparesis is a condition that slows or stops the movement of food out of the stomach and into the small intestines.\nThe condition can be caused by underlying medical issues, and one of the more common causes of gastroparesis is diabetes, according to the American College of Gastroenterology.          </td><td>The drugmakers Novo Nordisk and Eli Lilly failed to adequately warn patients about the possible risk of severe stomach problems associated with their blockbuster drugs Ozempic and Mounjaro, according to a lawsuit filed Wednesday.\n\nThe 26-page lawsuit, filed on behalf of a Louisiana woman who says she was “severely injured” after taking the two diabetes drugs, is the first to allege that they can cause gastrointestinal injuries.\n\nOzempic and Mounjaro — both part of a new class of medicines known as GLP-1 agonists — help people with Type 2 diabetes manage their blood sugar levels. They are also prescribed off-label for weight loss.\n\nGLP-1 medications work, in part, by slowing how quickly